### Importing libraties

In [105]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

### Define parameters

In [106]:
filepath = 'bumpgrid.dat.txt'

gamma = 1.4

rho_inf = 1.225 
Mach    = 1.4 
P_inf   = 1.0*1e+5
a_inf   = np.sqrt(gamma*P_inf/rho_inf)  # a_inf = sqrt(gamma p / rho) = 1
u_inf   = Mach*a_inf                   
v_inf   = 0.0       

CFL     = 0.5
tol     = 1e-4
niter   = 5000

eps = 1e-12


### Reading Dataset and creating geometry

In [107]:
data = np.loadtxt(filepath)
data = np.array(data)

[imx, jmx] = data[0, :]
imx = int(imx)
jmx = int(jmx)

X = np.zeros([imx, jmx])
Y = np.zeros([imx, jmx])

for j in range(jmx):
    for i in range(imx):
        X[i, j] = data[j * imx + i + 1, 0]
        Y[i, j] = data[j * imx + i + 1, 1]

In [108]:
xnx = np.zeros([imx + 1, jmx + 1])
xny = np.zeros([imx + 1, jmx + 1])

# i-direction faces
for j in range(1, jmx):
    for i in range(imx):
        xnx[i, j] = Y[i, j] - Y[i, j - 1]
        xny[i, j] = -(X[i, j] - X[i, j - 1])

ynx = np.zeros([imx + 1, jmx + 1])
yny = np.zeros([imx + 1, jmx + 1])

for j in range(jmx):
    for i in range(1, imx):
        ynx[i, j] = Y[i - 1, j] - Y[i, j]
        yny[i, j] = -(X[i - 1, j] - X[i, j])

vol = np.zeros([imx + 1, jmx + 1])

for j in range(1, jmx):
    for i in range(1, imx):
        x_1, y_1 = X[i - 1, j - 1], Y[i - 1, j - 1]
        x_2, y_2 = X[i, j - 1], Y[i, j - 1]
        x_3, y_3 = X[i, j], Y[i, j]
        x_4, y_4 = X[i - 1, j], Y[i - 1, j]
        vol[i, j] = 0.5 * (
            abs((x_2 - x_1) * (y_4 - y_2) - (y_2 - y_1) * (x_4 - x_1)) +
            abs((x_4 - x_3) * (y_2 - y_3) - (y_4 - y_3) * (x_2 - x_3))
        )


### Create functions to convert functions

In [109]:
def conservative_to_primitive(rho, rhou, rhov, rhoE, gamma=1.4):
    u = rhou/rho
    v = rhov/rho
    velocity = np.sqrt(u**2 + v**2)
    P = (gamma - 1) * (rhoE - 0.5 * rho * velocity**2)
    return u, v, P

def primitive_to_conservative(rho, u, v, P, gamma=1.4):
    # convert primitive variables to conservative variables
    velocity = np.sqrt(u**2 + v**2)
    # Use correct relation: internal energy per unit mass = P/((gamma - 1)*rho)
    E = P/((gamma - 1)*rho) + 0.5 * velocity**2
    return rho, rho*u, rho*v, rho*E


In [110]:
def Roe_Dissipation(q_L, q_R,n_x,n_y,gamma=1.4):

    Area = math.sqrt(n_x**2 + n_y**2)
    nx_hat = n_x/Area
    ny_hat = n_y/Area

    # Left State
    rho_L = q_L[0]
    ul = q_L[1]/rho_L
    vl = q_L[2]/rho_L
    Pl = (gamma - 1) * (q_L[3] - 0.5 * rho_L * (ul**2 + vl**2))
    h_L = (q_L[3] + Pl)/rho_L

    # Right State
    rho_R = q_R[0]
    ur = q_R[1]/rho_R
    vr = q_R[2]/rho_R
    Pr = (gamma - 1) * (q_R[3] - 0.5 * rho_R * (ur**2 + vr**2))
    h_R = (q_R[3] + Pr)/rho_R

    # Normal Velocities
    vnl = ul*nx_hat + vl*ny_hat
    vnr = ur*nx_hat + vr*ny_hat

    # Roe Averages
    r = max(math.sqrt(max(rho_R/rho_L,eps)),eps)
    rho_R = math.sqrt(max(rho_L*rho_R,eps))
    u_R = (ul + r*ur)/(1 + r)
    v_R = (vl + r*vr)/(1 + r)
    h0_R = (h_L + r*h_R)/(1 + r)
    # robust acoustic speed computation: avoid sqrt of tiny negative due to numerical noise
    a_R_sq = max((gamma - 1) * (h0_R - 0.5 * (u_R**2 + v_R**2)),eps)
    a_R = math.sqrt(a_R_sq)
    vn_R = u_R*nx_hat + v_R*ny_hat

    d_rho = rho_R - rho_L
    d_P = Pr - Pl
    d_vn = vnr - vnl
    d_u = ur - ul
    d_v = vr - vl

    # Construct alpha coefficients
    denom = max(a_R*a_R, eps)
    alpha_1 = Area*abs(vn_R)*(d_rho - d_P/denom)
    alpha_2 = Area/(2*denom)*abs(vn_R + a_R)*(d_P + rho_R*a_R*d_vn)
    alpha_3 = Area/(2*denom)*abs(vn_R - a_R)*(d_P - rho_R*a_R*d_vn)
    alpha_4 = alpha_1 + alpha_2 + alpha_3
    alpha_5 = a_R*(alpha_2-alpha_3)
    alpha_6 = Area*abs(vn_R)*(rho_R*d_u - nx_hat*rho_R*d_vn)
    alpha_7 = Area*abs(vn_R)*(rho_R*d_v - ny_hat*rho_R*d_vn)

    # Dissipation Fluxes
    d_1 = alpha_4
    d_2 = alpha_4*u_R + alpha_5*nx_hat + alpha_6
    d_3 = alpha_4*v_R + alpha_5*ny_hat + alpha_7
    d_4 = alpha_4*h0_R + alpha_5*vn_R + alpha_6*u_R + alpha_7*v_R - a_R*alpha_1/(gamma - 1)

    return np.array([d_1, d_2, d_3, d_4])

### Function for computing fluxes on face normals

In [111]:
def central_flx(q,nx,ny,gamma=1.4):

    # Will compute the central flux across a face with normal (nx, ny)

    A = math.sqrt(nx**2 + ny**2)
    nx_hat = nx/A
    ny_hat = ny/A

    rho = q[0]
    u = q[1]/rho
    v = q[2]/rho
    P = (gamma - 1) * (q[3] - 0.5 * rho * (u**2 + v**2))

    vn = u*nx_hat + v*ny_hat

    F1 = rho*vn*A
    F2 = (rho*u*vn + P*nx_hat)*A
    F3 = (rho*v*vn + P*ny_hat)*A
    F4 = (q[3] + P)*vn*A

    return np.array([F1, F2, F3, F4])


In [112]:
def save_solution(iter_num, rho, rhou, rhov, rhoE, gamma, X, Y):
    """Save primitive solution (rho, u, v, p) at nodes to CSV."""
    imx, jmx = X.shape
    u_node = np.zeros((imx, jmx))
    v_node = np.zeros((imx, jmx))
    p_node = np.zeros((imx, jmx))
    rho_node = np.zeros((imx, jmx))

    # cell-centered values are at [1:imx,1:jmx] in arrays of size (imx+1,jmx+1)
    for j in range(jmx):
        for i in range(imx):
            # average of surrounding 4 cells
            r_c = 0.25 * (rho[i, j] + rho[i + 1, j] +
                          rho[i, j + 1] + rho[i + 1, j + 1])
            ru_c = 0.25 * (rhou[i, j] + rhou[i + 1, j] +
                           rhou[i, j + 1] + rhou[i + 1, j + 1])
            rv_c = 0.25 * (rhov[i, j] + rhov[i + 1, j] +
                           rhov[i, j + 1] + rhov[i + 1, j + 1])
            rE_c = 0.25 * (rhoE[i, j] + rhoE[i + 1, j] +
                           rhoE[i, j + 1] + rhoE[i + 1, j + 1])

            uc, vc, pc = conservative_to_primitive(r_c, ru_c, rv_c, rE_c, gamma)
            u_node[i, j] = uc
            v_node[i, j] = vc
            p_node[i, j] = pc
            rho_node[i, j] = r_c

    output_data = []
    for j in range(jmx):
        for i in range(imx):
            output_data.append([
                X[i, j], Y[i, j],
                rho_node[i, j],
                u_node[i, j],
                v_node[i, j],
                p_node[i, j]
            ])
    df = pd.DataFrame(output_data,
                      columns=['x', 'y', 'rho', 'u', 'v', 'p'])
    filename = f"solution_roe_iter_{iter_num}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved solution to {filename}")


### Defining the Conservative variables as well as the scales

In [113]:
rho = rho_inf*np.ones([imx + 1, jmx + 1])
u = u_inf*np.ones([imx + 1, jmx + 1])
v = v_inf*np.ones([imx + 1, jmx + 1])
P = P_inf*np.ones([imx + 1, jmx + 1])

rho, rhou, rhov, rhoE = primitive_to_conservative(rho, u, v, P, gamma)

res = np.zeros([4, imx + 1, jmx + 1])
delta_t = np.zeros([imx + 1, jmx + 1])

residal_norms = []

# Scaling for the residuals

scale_r = np.zeros([4])
scale_r[0] = rho_inf*u_inf
scale_r[1] = rho_inf*u_inf**2
scale_r[2] = rho_inf*u_inf**2
scale_r[3] = rho_inf*u_inf*((gamma/(gamma - 1))*P_inf/rho_inf + 0.5*u_inf**2)

### Computing of the residuals

In [114]:
def compute_residuals(rho, rhou, rhov, rhoE, imx=imx, jmx=jmx, xnx=xnx, xny=xny, ynx=ynx, yny=yny, vol=vol, gamma=gamma):

    # computing primitive variables
    u, v, P = conservative_to_primitive(rho, rhou, rhov, rhoE, gamma)

    # Boundary condiiitions

    # At the inlet (left boundary)
    if Mach > 1:
        rho[0, :] = rho_inf
        u[0, :] = u_inf
        v[0, :] = v_inf
        P[0, :] = P_inf
    else:
        rho[0, :] = rho[1, :]
        u[0, :] = u[1, :]
        v[0, :] = v[1, :]
        P[0, :] = P[1, :]

    rho[0, :], rhou[0, :], rhov[0, :], rhoE[0, :] = primitive_to_conservative(rho[0, :], u[0, :], v[0, :], P[0, :], gamma)

    # At the outlet (right boundary) - extrapolation

    rho[imx, :] = rho[imx - 1, :]
    u[imx, :] = u[imx - 1, :]
    v[imx, :] = v[imx - 1, :]
    P[imx, :] = P[imx - 1, :]

    # Bottom Wall

    j = 0
    for i in range(imx + 1):
        area = math.sqrt(ynx[i, j]**2 + yny[i, j]**2) + eps
        v_n = u[i, j+1]*ynx[i, j]/area + v[i, j+1]*yny[i, j]/area
        rho[i, j] = rho[i, j+1]
        u[i, j] = u[i, j+1] - 2*v_n*ynx[i, j]/area
        v[i, j] = v[i, j+1] - 2*v_n*yny[i, j]/area
        P[i, j] = P[i, j+1]
        
        rho[i, j], rhou[i, j], rhov[i, j], rhoE[i, j] = primitive_to_conservative(rho[i, j], u[i, j], v[i, j], P[i, j], gamma)

    # Top Wall

    j = jmx
    for i in range(imx + 1):
        area = math.sqrt(ynx[i, j - 1]**2 + yny[i, j - 1]**2) + eps
        v_n = u[i, j - 1]*ynx[i, j - 1]/area + v[i, j - 1]*yny[i, j - 1]/area
        rho[i, j] = rho[i, j - 1]
        u[i, j] = u[i, j - 1] - 2*v_n*ynx[i, j - 1]/area
        v[i, j] = v[i, j - 1] - 2*v_n*yny[i, j - 1]/area
        P[i, j] = P[i, j - 1]
        
        rho[i, j], rhou[i, j], rhov[i, j], rhoE[i, j] = primitive_to_conservative(rho[i, j], u[i, j], v[i, j], P[i, j], gamma)

    # Making the flux storage zero
    xflux = np.zeros([4, imx + 1, jmx + 1])
    yflux = np.zeros([4, imx + 1, jmx + 1])

    # i-direction faces
    for j in range(1, jmx):
        for i in range(imx):
            q_L = np.array([rho[i, j], rhou[i, j], rhov[i, j], rhoE[i, j]])
            q_R = np.array([rho[i+1, j], rhou[i+1, j], rhov[i+1, j], rhoE[i+1, j]])

            f_c = 0.5*(central_flx(q_L, xnx[i, j], xny[i, j], gamma) + central_flx(q_R, xnx[i, j], xny[i, j], gamma))
            f_d = Roe_Dissipation(q_L, q_R, xnx[i, j], xny[i, j], gamma)

            xflux[:, i, j] = f_c - 0.5*f_d

    # j-direction faces
    for j in range(jmx):
        for i in range(1, imx):
            q_L = np.array([rho[i, j], rhou[i, j], rhov[i, j], rhoE[i, j]])
            q_R = np.array([rho[i, j+1], rhou[i, j+1], rhov[i, j+1], rhoE[i, j+1]])

            f_c = 0.5*(central_flx(q_L, ynx[i, j], yny[i, j], gamma) + central_flx(q_R, ynx[i, j], yny[i, j], gamma))
            f_d = Roe_Dissipation(q_L, q_R, ynx[i, j], yny[i, j], gamma)

            yflux[:, i, j] = f_c - 0.5*f_d

    # Computing the residuals
    res = np.zeros([4, imx + 1, jmx + 1])
    resnorm = 0.0

    for j in range(1, jmx):
        for i in range(1, imx):
            res[:, i, j] = res[:, i, j] + xflux[:, i, j] - xflux[:, i-1, j] + yflux[:, i, j] - yflux[:, i, j-1]

            # Computing the residual norm
            for var in range(4):
                resnorm = resnorm + (res[var, i, j]/(scale_r[var]+eps))**2

    return res, resnorm

### Time step calculation

In [115]:
def compute_dt_over_vol(rho, rhou, rhov, rhoE):
    """
    Compute local dt/vol using the same CFL formula you had before,
    based on current primitive variables.
    """
    u = rhou / rho
    v = rhov / rho
    vel2 = u * u + v * v
    p = (gamma - 1.0) * (rhoE - 0.5 * rho * vel2)

    delt_local = np.zeros_like(rho)

    for j in range(1, jmx):
        for i in range(1, imx):
            # left face
            nx_l = xnx[i - 1, j]
            ny_l = xny[i - 1, j]
            A_l = math.sqrt(nx_l * nx_l + ny_l * ny_l) + eps
            u_l = u[i, j]
            v_l = v[i, j]
            p_l = p[i, j]
            a_l = math.sqrt(max(gamma * p_l / rho[i, j], 0.0))
            vn_l = (u_l * nx_l + v_l * ny_l) / A_l
            lam_l = A_l * (abs(vn_l) + a_l)

            # right face
            nx_r = xnx[i, j]
            ny_r = xny[i, j]
            A_r = math.sqrt(nx_r * nx_r + ny_r * ny_r) + eps
            vn_r = (u[i, j] * nx_r + v[i, j] * ny_r) / A_r
            lam_r = A_r * (abs(vn_r) + a_l)

            # bottom face
            nx_b = ynx[i, j - 1]
            ny_b = yny[i, j - 1]
            A_b = math.sqrt(nx_b * nx_b + ny_b * ny_b) + eps
            vn_b = (u[i, j] * nx_b + v[i, j] * ny_b) / A_b
            lam_b = A_b * (abs(vn_b) + a_l)

            # top face
            nx_t = ynx[i, j]
            ny_t = yny[i, j]
            A_t = math.sqrt(nx_t * nx_t + ny_t * ny_t) + eps
            vn_t = (u[i, j] * nx_t + v[i, j] * ny_t) / A_t
            lam_t = A_t * (abs(vn_t) + a_l)

            lam_sum = lam_l + lam_r + lam_b + lam_t + eps
            delt_local[i, j] = CFL * 2.0 * vol[i, j] / lam_sum

    # Convert to dt_over_vol for convenience
    dt_over_vol = np.zeros_like(rho)
    for j in range(1, jmx):
        for i in range(1, imx):
            dt_over_vol[i, j] = delt_local[i, j] / max(vol[i, j], eps)

    return dt_over_vol

In [116]:
residual_norm = []


In [117]:
for it in range(niter):

    # compute local dt/vol once from current state
    dt_over_vol = compute_dt_over_vol(rho, rhou, rhov, rhoE)

    # -------- Stage 1 --------
    # work on copies because compute_residual modifies arrays via BCs
    rho1  = rho.copy()
    rhou1 = rhou.copy()
    rhov1 = rhov.copy()
    rhoE1 = rhoE.copy()

    res1, resnorm = compute_residuals(rho1, rhou1, rhov1, rhoE1)

    if it == 0:
        resnorm0 = resnorm if resnorm > 0 else 1.0

    resnorm_rel = resnorm / resnorm0
    residual_norm.append(resnorm_rel)

    if it % 10 == 0:
        print(f"Iteration {it}: Residual = {resnorm_rel:.6e}")

    if resnorm_rel < tol:
        print(f"Converged at iteration {it} with residual {resnorm_rel:.6e}")
        break

    # -------- Stage 2 --------
    rho2  = rho.copy()
    rhou2 = rhou.copy()
    rhov2 = rhov.copy()
    rhoE2 = rhoE.copy()

    for j in range(1, jmx):
        for i in range(1, imx):
            rho2[i, j]  -= 0.5 * dt_over_vol[i, j] * res1[0, i, j]
            rhou2[i, j] -= 0.5 * dt_over_vol[i, j] * res1[1, i, j]
            rhov2[i, j] -= 0.5 * dt_over_vol[i, j] * res1[2, i, j]
            rhoE2[i, j] -= 0.5 * dt_over_vol[i, j] * res1[3, i, j]

    rho2c  = rho2.copy()
    rhou2c = rhou2.copy()
    rhov2c = rhov2.copy()
    rhoE2c = rhoE2.copy()
    res2, _ = compute_residuals(rho2c, rhou2c, rhov2c, rhoE2c)

    # -------- Stage 3 --------
    rho3  = rho.copy()
    rhou3 = rhou.copy()
    rhov3 = rhov.copy()
    rhoE3 = rhoE.copy()

    for j in range(1, jmx):
        for i in range(1, imx):
            rho3[i, j]  -= 0.5 * dt_over_vol[i, j] * res2[0, i, j]
            rhou3[i, j] -= 0.5 * dt_over_vol[i, j] * res2[1, i, j]
            rhov3[i, j] -= 0.5 * dt_over_vol[i, j] * res2[2, i, j]
            rhoE3[i, j] -= 0.5 * dt_over_vol[i, j] * res2[3, i, j]

    rho3c  = rho3.copy()
    rhou3c = rhou3.copy()
    rhov3c = rhov3.copy()
    rhoE3c = rhoE3.copy()
    res3, _ = compute_residuals(rho3c, rhou3c, rhov3c, rhoE3c)

    # -------- Stage 4 --------
    rho4  = rho.copy()
    rhou4 = rhou.copy()
    rhov4 = rhov.copy()
    rhoE4 = rhoE.copy()

    for j in range(1, jmx):
        for i in range(1, imx):
            rho4[i, j]  -= dt_over_vol[i, j] * res3[0, i, j]
            rhou4[i, j] -= dt_over_vol[i, j] * res3[1, i, j]
            rhov4[i, j] -= dt_over_vol[i, j] * res3[2, i, j]
            rhoE4[i, j] -= dt_over_vol[i, j] * res3[3, i, j]

    rho4c  = rho4.copy()
    rhou4c = rhou4.copy()
    rhov4c = rhov4.copy()
    rhoE4c = rhoE4.copy()
    res4, _ = compute_residuals(rho4c, rhou4c, rhov4c, rhoE4c)

    # -------- Final RK4 combination --------
    for j in range(1, jmx):
        for i in range(1, imx):
            factor = dt_over_vol[i, j] / 6.0
            rho[i, j]  -= factor * (res1[0, i, j] + 2.0 * res2[0, i, j] +
                                    2.0 * res3[0, i, j] + res4[0, i, j])
            rhou[i, j] -= factor * (res1[1, i, j] + 2.0 * res2[1, i, j] +
                                    2.0 * res3[1, i, j] + res4[1, i, j])
            rhov[i, j] -= factor * (res1[2, i, j] + 2.0 * res2[2, i, j] +
                                    2.0 * res3[2, i, j] + res4[2, i, j])
            rhoE[i, j] -= factor * (res1[3, i, j] + 2.0 * res2[3, i, j] +
                                    2.0 * res3[3, i, j] + res4[3, i, j])

Iteration 0: Residual = 1.000000e+00
Iteration 10: Residual = 2.078561e-01
Iteration 10: Residual = 2.078561e-01
Iteration 20: Residual = 1.394214e-01
Iteration 20: Residual = 1.394214e-01
Iteration 30: Residual = 1.146469e-01
Iteration 30: Residual = 1.146469e-01
Iteration 40: Residual = 7.536070e+267
Iteration 40: Residual = 7.536070e+267


/tmp/ipykernel_68719/198286909.py:91: RuntimeWarning: overflow encountered in scalar power
  resnorm = resnorm + (res[var, i, j]/(scale_r[var]+eps))**2
/tmp/ipykernel_68719/2910475452.py:27: RuntimeWarning: overflow encountered in scalar multiply
  rho_R = math.sqrt(max(rho_L*rho_R,eps))
/tmp/ipykernel_68719/2910475452.py:47: RuntimeWarning: invalid value encountered in scalar add
  alpha_4 = alpha_1 + alpha_2 + alpha_3
/tmp/ipykernel_68719/2910475452.py:49: RuntimeWarning: invalid value encountered in scalar subtract
  alpha_6 = Area*abs(vn_R)*(rho_R*d_u - nx_hat*rho_R*d_vn)
/tmp/ipykernel_68719/2910475452.py:50: RuntimeWarning: invalid value encountered in scalar multiply
  alpha_7 = Area*abs(vn_R)*(rho_R*d_v - ny_hat*rho_R*d_vn)
/tmp/ipykernel_68719/2910475452.py:55: RuntimeWarning: invalid value encountered in scalar multiply
  d_3 = alpha_4*v_R + alpha_5*ny_hat + alpha_7
/tmp/ipykernel_68719/2860786303.py:19: RuntimeWarning: overflow encountered in scalar multiply
  F4 = (q[3] + P

Iteration 50: Residual = nan


KeyboardInterrupt: 